<a href="https://www.kaggle.com/code/mehransaifi/crime-against-women-in-india-analysis?scriptVersionId=205743017" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
df = pd.read_csv('/kaggle/input/crimes-against-women-in-india-a-20-year-analysis/CrimesOnWomenData.csv')

df.head()

In [ ]:
df.info()

In [ ]:
# CONVERTING THE STATE COLUMN IN LOWER CASE SO THAT EVERY STATE NAME TREATED EQUALLY
df['State'] = df['State'].str.lower() # IN DATA FRAME STATE COLUMN HAS MIXED NAME VALUES LIKE UTTAR PRADESH & uttar pradesh THAT'S WHY ITS IMPORTANT TO CONVERT ALL THE STATE VALUES IN LOWER CASE.....


In [ ]:
# Replace the incorrect state name with the correct one
df['State'] = df['State'].replace('d & n haveli', 'd&n haveli')


In [ ]:
# COLUMNS DESCRIPTION DATA FOR THE REFRENCE..........
desc = pd.read_csv('/kaggle/input/crimes-against-women-in-india-a-20-year-analysis/description.csv')

desc.iloc[:,[1,2]] 

# GENERATING THE EDA REPORT FOR THE DATASET

In [ ]:
# GENERATING BASIC EDA REPORT.......
profile = ProfileReport(df , dark_mode=True , title= 'EDA REPORT' , explorative=True)

profile

In [ ]:
df.columns

# LINE CHART FOR ALL THE STATE & FOR ALL CRIMES

In [ ]:
# LINE CHART TO LOOK AT THE EVERY CRIME RATE OVER TIME FOR EVERY STATE..........

# Define the columns for different crimes and states
col = df[['Rape','K&A', 'DD', 'AoW', 'AoM', 'DV', 'WT']]

# Create a list of unique states
states = df['State'].unique()

# Create a figure using go.Figure()
fig = go.Figure()

# Initialize visibility with all traces off
visibility = {state: {crime: False for crime in col.columns} for state in states}
visibility[states[0]]['Rape'] = True  # Make the first crime for the first state visible by default

# Add traces for each crime type and state, but keep them invisible except for the default state and crime
for state in states:
    for crime in col.columns:
        fig.add_trace(
            go.Scatter(x=df[df['State'] == state]['Year'], 
                       y=df[df['State'] == state][crime], 
                       mode='lines',
                       name=f"{crime} - {state}",
                       visible=visibility[state][crime])
        )


# Create a dropdown menu for state selection
state_dropdown_buttons = [
    {'label': state, 
     'method': 'update', 
     'args': [
         {'visible': [(trace.name.endswith(state)) for trace in fig.data]}, 
         {'title': f'State: {state}'}
     ]}
    for state in states
]

# Update the layout to include both dropdowns (crime and state)
fig.update_layout(
    updatemenus=[
        {
            'buttons': state_dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.05,  # Adjust position of the dropdown
            'xanchor': 'left',
            'y': 1.15,
            'yanchor': 'top',
            'pad': {'r': 10, 't': 10}
        }
    ],
    title="Select State",
    xaxis_title="Year",
    yaxis_title="Crime Rate",
)

# Show the figure
fig.show()

# WHICH CRIME IS AT THE TOP IN INDIA

In [ ]:
# Define the columns for different crimes
crime_columns = ['Rape', 'K&A', 'DD', 'AoW', 'AoM', 'DV', 'WT']

# Sum the crime columns across all states to get total crime counts for India
crime_totals = df[crime_columns].sum()

# Create a DataFrame from the totals for better visualization
crime_totals_df = pd.DataFrame({'Crime': crime_totals.index, 'Total': crime_totals.values})

# Create a bar chart using Plotly Express to visualize which crime is at the top
fig = px.bar(crime_totals_df.sort_values(by='Total', ascending=False), x='Crime', y='Total', title='Total Crimes in India by Type',
             labels={'Total': 'Total Count', 'Crime': 'Crime Type'}, 
             color='Crime')

# Show the bar chart
fig.show()

# IN WHICH STATE WHICH CRIME IS HIGHEST 

In [ ]:
# Define the columns for different crimes
crime_columns = ['Rape', 'K&A', 'DD', 'AoW', 'AoM', 'DV', 'WT']

# Create a figure object to hold all crime plots
figures = []

# Loop through each crime and find the top 10 states for that crime
for crime in crime_columns:
    # Group by state and sum the crime data, then sort to get the top 10 states
    top_10_states = df.groupby('State')[crime].sum().nlargest(37).reset_index()
    
    # Create a bar chart for the current crime
    fig = px.bar(top_10_states, x='State', y=crime, title=f'All States for {crime} in India',
                 labels={crime: 'Total Count', 'State': 'State'}, color='State',
                 text=crime, height=800)
    
    # Append the figure to the list of figures
    figures.append(fig)

# Show the charts (one for each crime)
for fig in figures:
    fig.show()

# CRIME WISE RANKING OF EACH STATE.......

In [ ]:
# Define the columns for different crimes
crime_columns = ['Rape', 'K&A', 'DD', 'AoW', 'AoM', 'DV', 'WT']

# Sum all crime columns for each state to get the total crime count per state
df['Total_Crime'] = df[crime_columns].sum(axis=1)

# Group by state and sum the total crime count
state_crime_totals = df.groupby('State')['Total_Crime'].sum().reset_index()

# Rank the states by total crime count (descending order)
state_crime_totals['Rank'] = state_crime_totals['Total_Crime'].rank(ascending=False, method='min')

# Sort the DataFrame by total crime count to show the highest at the top
state_crime_totals = state_crime_totals.sort_values(by='Total_Crime', ascending=False).reset_index(drop=True)

# Display the ranked list of states by total crime
print(state_crime_totals)

# Create a bar chart to visualize the total crimes per state
fig = px.bar(state_crime_totals, x='State', y='Total_Crime', title='Total Crimes by State in India (Overall Ranking)',
             labels={'Total_Crime': 'Total Crime Count', 'State': 'State'}, 
             text='Rank', color='State' , height = 800)

# Show the bar chart
fig.show()

# EVERY CRIME RATIO IN EACH STATE............

In [ ]:
# Define the columns for different crimes
crime_columns = ['Rape', 'K&A', 'DD', 'AoW', 'AoM', 'DV', 'WT']

# Aggregate the data by state
state_crime_totals = df.groupby('State')[crime_columns].sum().reset_index()

# Number of states
num_states = len(state_crime_totals)

# Define the number of columns and rows for the subplots
num_cols = 4  # You can adjust this based on your preference
num_rows = (num_states + num_cols - 1) // num_cols  # Compute rows needed

# Create subplots
fig = make_subplots(rows=num_rows, cols=num_cols, 
                    subplot_titles=state_crime_totals['State'],
                    specs=[[{'type': 'pie'}] * num_cols] * num_rows)

# Add pie charts for each state to the subplots
for idx, state in enumerate(state_crime_totals['State']):
    state_data = state_crime_totals[state_crime_totals['State'] == state].iloc[0]
    
    row = idx // num_cols + 1
    col = idx % num_cols + 1
    
    pie_chart = go.Pie(
        labels=crime_columns,
        values=state_data[crime_columns],
        name=state
    )
    
    fig.add_trace(pie_chart, row=row, col=col)

# Update layout for better spacing and titles
fig.update_layout(
    title_text='Crime Distribution by State',
    showlegend=False,  # Hide legends to avoid cluttering
    height=600 + (num_rows - 1) * 300 # Adjust height based on number of rows  # Adjust width as needed
)

# Show the figure
fig.show()

# THANK YOU .............